In [1]:
import pandas as pd
import requests
import sqlite3
from plyer import notification
from datetime import datetime

In [43]:
def save_to_db(df, table_name, conn):
    df.to_sql(table_name, conn, if_exists='replace', index=False)
    print(f"Tabela {table_name} salva no banco de dados")

Pokemons

In [49]:
url = "https://pokeapi.co/api/v2/pokemon"
response = requests.get(url)

In [ ]:
if response.status_code == 200:
    data_json = response.json()
    print("Funcionou")

    pokemon_list = []
    for pokemon in data_json['results']:
        pokemon_url = pokemon['url']
        pokemon_response = requests.get(pokemon_url)
        if pokemon_response.status_code == 200:
            pokemon_data = pokemon_response.json()            
        
            types = ", ".join([type_info['type']['name'] for type_info in pokemon_data['types']])
            
            pokemon_info = {
                'name': pokemon_data['name'],
                'height': pokemon_data['height'],
                'weight': pokemon_data['weight'],
                'types': types,
                'base_experience': pokemon_data['base_experience']
            }
            pokemon_list.append(pokemon_info)
        else:
            print(f"Falha ao acessar dados de {pokemon['name']}")

    df_pokemons = pd.DataFrame(pokemon_list)
else:
    print("Não foi")

In [ ]:
df_pokemons.duplicated()
df_pokemons.drop_duplicates()

In [ ]:
df_pokemons.groupby('height')
df_pokemons.sort_values('height', ascending=True)

In [ ]:
conn = sqlite3.connect('coderhouse.db')
save_to_db(df_pokemons, 'Pokemon', conn)
conn.close()

In [ ]:
conn = sqlite3.connect('coderhouse.db')
query = "SELECT * FROM Pokemon"
df = pd.read_sql(query, conn)
print(df)

conn.close()

TMs

In [68]:
url = "https://pokeapi.co/api/v2/machine"
response = requests.get(url)

In [ ]:
if response.status_code == 200:
    data_json = response.json()
    print("Funcionou")

    machine_list = []
    for machine in data_json['results']:
        machine_url = machine['url']
        machine_response = requests.get(machine_url)
        machine_data = machine_response.json()
        machine_info = {
             'id': machine_data['id'],
             'TM': machine_data['item']['name'],
             'generation': machine_data['version_group']['name'],
             'move': machine_data['move']['name'],

            }
        machine_list.append(machine_info)
    df_machine_list = pd.DataFrame(machine_list)
    
else:
    print("Não foi")

In [ ]:
df_machine_list.duplicated()
df_machine_list.drop_duplicates()

In [ ]:
conn = sqlite3.connect('coderhouse.db')
save_to_db(df_machine_list, 'TMs', conn)
conn.close()

In [ ]:
conn = sqlite3.connect('coderhouse.db')
query = "SELECT * FROM TMs"
df = pd.read_sql(query, conn)
print(df)

conn.close()

Moves

In [85]:
url = "https://pokeapi.co/api/v2/move"
response = requests.get(url)

In [ ]:
if response.status_code == 200:
    data_json = response.json()
    print("funcionou")

    move_list = []
    for move in data_json ['results']:
        move_url = move['url']
        move_response = requests.get(move_url)
        move_data = move_response.json()
        move_info = {
            'name': move_data ['name'],
            'type': move_data ['type']['name'],
            'power': move_data['power'],
            'accuracy': move_data['accuracy'],
            'power points': move_data['pp'],
            'effect': move_data ['effect_entries'][0]['short_effect'],
        }
        move_list.append(move_info)
    df_moves = pd.DataFrame(move_list)

else:
    print("nao funcionou")

In [ ]:
df_moves.duplicated()
df_moves.drop_duplicates

In [ ]:
conn = sqlite3.connect('coderhouse.db')
save_to_db(df_moves, 'Moves', conn)
conn.close()

In [ ]:
conn = sqlite3.connect('coderhouse.db')
query = "SELECT * FROM Moves"
df = pd.read_sql(query, conn)
print(df)

conn.close()

Notification

In [12]:
def alerta(nivel, base, etapa):
    now = str(datetime.now())
    msg = f"Falha no carregamento da base {base} na etapa {etapa}.\n{now}"

    if nivel == 1:
        title = 'ATENÇÃO: Alerta Baixo'
    elif nivel == 2:
        title = 'ATENÇÃO: Alerta Médio'
    elif nivel  == 3:
        title = 'ATENÇÃO: Alerta Alto'
    else:
        print("Nivel",nivel,"não disponível!")

    notification.notify(
            title=title,
            message=msg,
            app_name='alerta',
            timeout=10
        )

alerta(nivel = 3,
        base = "POKEDEX", 
        etapa = "EXTRACAO")

In [ ]:
conn = sqlite3.connect('coderhouse.db')
query = "SELECT name FROM sqlite_master WHERE type='table'"
schema = pd.read_sql_query(query, conn)
print(schema)

conn.close()